### Import Libraries

In [ ]:
#Import libraries
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#read csv file & assign to a dataframe from https://www.kaggle.com/karthickveerakumar/spam-filter/version/1#emails.csv
df = pd.read_csv("../input/spam-filter/emails.csv")

In [ ]:
#show the first 5 elements of the csv
df.head(10)

In [ ]:
#print number of rows and columns
df.shape

In [ ]:
df.info()

In [ ]:
#show element n
print(df.iloc[100][0])


In [ ]:
df.isnull().sum() #shows how many rows are null, this is good practice if we want to take care of null value in data prep

In [ ]:
#In Python3, string.punctuation is a pre-initialized string used as string constant, this contain all the punctuation
punct= string.punctuation

In [ ]:
punct

In [ ]:
#nltk.download('stopwords')
#this package from natural language toolkit contains all the stopword in the major languages

In [ ]:
stop_words = set(stopwords.words("italian"))

In [ ]:
stop_words

### Clean the DataFrame

In [ ]:
#now we remove all punctuations and stop words
df

In [ ]:
def remove_punct(text):
    nopunct = [char for char in text if char not in punct]
    nopunct = ''.join(nopunct)
    return(nopunct)

#this function remove the punctuation
    

In [ ]:
df1 = df['text'].apply(remove_punct)

In [ ]:
df1

In [ ]:
#now we remove all stop words
def remove_stop(text):
    nostop = [word for word in text.split() if word.lower() not in stop_words]
    return(nostop)

In [ ]:
df2 = df1.apply(remove_stop)

In [ ]:
df2 #at this point we have a list of tokens to analyze

In [ ]:
def remove_everything(text):
    return remove_stop(remove_punct(text))

#this is definetely not the best coding - but is just to show the progression

In [ ]:
df['text'].apply(remove_everything)

### Example of transforming a string into a countvector

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
message = 'KEA is a university, /////located in Nørrebro I love burger burger'
message2 = "are great or burger, burger"
message3 = 'hey i love berries and burger love burger a lot burger'
print(message)

In [ ]:
vectorizer = CountVectorizer(analyzer=remove_everything)
#initialize a vector object


In [ ]:
vect_fit_trans = vectorizer.fit_transform([message,message2,message3]) 
#run the function fit_transform over the array of messages

In [ ]:
vectorizer.get_feature_names()
#shows all the features extracted - they are simply put in alphabetical order

In [ ]:
print(vect_fit_trans)
#each word is transformed into a vector (x,y) n
#bag of words

### Transform the entire df

In [ ]:
transformvc = CountVectorizer(analyzer=remove_everything)

In [ ]:
FiTrannsformvc = transformvc.fit_transform(df['text'])

In [ ]:
transformvc.get_feature_names()


### Split the Dataset

In [ ]:
#now we split the dataset 80% for training and 20% for test
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(FiTrannsformvc, df['spam'], test_size=0.20, random_state=75)

### Train the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# I used Naive Bayes, being one of the most popular algoryth for spam detection
# https://www.quora.com/What-are-the-popular-ML-algorithms-for-email-spam-detection

In [ ]:
X_train.shape #verify that train & test have the same shape, we fit_transform before the split, therefore it shouldn't be a problem

In [ ]:
X_test.shape

In [ ]:
print(classifier.predict(X_train))
print(y_train.values)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
#Print the predictions
print('Predicted value: ',classifier.predict(X_test))

#Print Actual Label
print('Actual value: ',y_test.values)

In [ ]:
#here we evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test ,pred ))


print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

In [ ]:
type(X_test)
X_test.shape


### try on a personal email

In [ ]:
realmsg = '''
INSERT A MESSAGE HERE
'''
print(realmsg)

In [ ]:
realvector = transformvc.transform([realmsg])

In [ ]:
type(realvector)
realvector.shape


In [ ]:
confidence = classifier.predict_proba(realvector)
prediction = classifier.predict(realvector)


In [ ]:
print(confidence[0,1])

In [ ]:
if confidence[0,1] > 0.8:
    print("This e-mail is Spam")
elif (confidence[0,1] > 0.5) and (confidence[0,1] < 0.8):
    print("This e-mail seems to be Spam, check it")
else:
    print("This e-mail is legit")

In [ ]:
print(prediction)